# Flag EOA that have less than 30 tx and a bot behavior

In [10]:
import os
from pathlib import Path
import numpy as np
import requests
import pandas as pd

In [11]:
import json

In [12]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [13]:
# Set path to data folder
current_dir = Path(os.getcwd())
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')
DATA_DIR_GITCOIN = os.path.join(current_dir.parent.parent, 'data-gitcoin')
GRANT_ROUND_ID = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
ODC_DATA_DIR = os.path.join(current_dir.parent.parent.parent, 'data')

PATH_TO_VOTES = os.path.join(DATA_DIR_GITCOIN, 'citizen-votes.csv')
# PATH_TO_GRANTS = os.path.join(DATA_DIR, "all-allo-rounds.csv")
# PATH_TO_PROJECTS = os.path.join(DATA_DIR, "projects_QmQurt.csv")

In [14]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(current_dir.parent.parent, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [15]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_8016\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [16]:
# Load tx data
CONTRACT_CREATION_AD_NAME = '0x_contract_creation'

df_tx = pd.read_csv(os.path.join(PATH_TO_EXPORT, tx_file_name))
df_tx.loc[df_tx['to_address'].isna(), 'to_address'] = CONTRACT_CREATION_AD_NAME

assert df_tx.isna().sum().sum() == 0
df_tx.drop(columns=['__row_index'], inplace=True)

print(f'df transaction shape : {df_tx.shape}')

# add EOA to df_tx
df_tx_EOA_FROM = df_tx.copy()
df_tx_EOA_FROM['EOA'] = df_tx_EOA_FROM['from_address']

df_tx_EOA_TO = df_tx.copy()
df_tx_EOA_TO['EOA'] = df_tx_EOA_TO['to_address']

df_tx_eoa = pd.concat([df_tx_EOA_FROM, df_tx_EOA_TO])
df_tx_eoa = df_tx_eoa.loc[df_tx_eoa['EOA'] != CONTRACT_CREATION_AD_NAME, :]



df transaction shape : (6894506, 8)


In [17]:
df_tx_eoa.shape

(13787932, 9)

In [18]:
tx_analyser = txa(df_tx_eoa.copy(), votes['voter'].str.lower().unique())

print(f'Len of df tx: {len(tx_analyser.df_transactions)}')

Len of df tx: 6953161


In [19]:
df_features = tx_analyser.gb_EOA_sorted['tx_hash'].count().reset_index().rename(columns={'tx_hash': 'count_tx'})

In [20]:
min_tx = 100
df_features_less_tx = df_features.loc[df_features['count_tx'] < min_tx, :]
print(f'Number of EOA with less than {min_tx} tx: {len(df_features_less_tx)}')

Number of EOA with less than 100 tx: 28825


Call the addres-tag API to get the tag of the address

In [21]:
address_list = df_features_less_tx['EOA'].unique()

In [39]:
sql_template = """
with dai as (
    SELECT
        tx_hash,
        block_timestamp,
        origin_from_address as voter,
        'DAI' as token,
        amount_usd
    from
        optimism.core.ez_token_transfers
    where
        block_timestamp between '2023-08-15 12:00:00.000'
        and '2023-08-29 12:00:00.000'
        and contract_address = '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1'
        and origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
    ),
    eth_time AS (
    SELECT
        tx_hash,
        block_timestamp,
        origin_from_address as voter,
        amount_usd,
    origin_to_address,
    eth_to_address
    from
        optimism.core.ez_eth_transfers
    where
        block_timestamp between '2023-08-15 12:00:00.000'
        and '2023-08-29 12:00:00.000'
    ),
    eth_direct as (
    SELECT
        tx_hash,
        block_timestamp,
        voter,
        'ETH' as token,
        amount_usd
    FROM
        eth_time
    WHERE
        origin_to_address IN (
        '0x8de918f0163b2021839a8d84954dd7e8e151326d',
        '0xb6be0ecafdb66dd848b0480db40056ff94a9465d',
        '0x2871742b184633f8dc8546c6301cbc209945033e',
        '0x10be322de44389ded49c0b2b73d8c3a1e3b6d871',
        '0x5b95acf46c73fd116f0fedadcbef453530e35d0',
        '0xc5fdf5cff79e92fac1d6efa725c319248d279200',
        '0xf591e42dffe8e62c2085ccaadfe05f84d89d0c6',
        '0x9331fde4db7b9d9d1498c09d30149929f24cf9d5',
        '0x30c381033aa2830ceb0aa372c2e4d28f004b3db9',
        '0x69e423181f1d3e6bebf8ab88030c36da73785f26'
        )
        and eth_to_address IN (
        '0x8de918f0163b2021839a8d84954dd7e8e151326d',
        '0xb6be0ecafdb66dd848b0480db40056ff94a9465d',
        '0x2871742b184633f8dc8546c6301cbc209945033e',
        '0x10be322de44389ded49c0b2b73d8c3a1e3b6d871',
        '0x5b95acf46c73fd116f0fedadcbef453530e35d0',
        '0xc5fdf5cff79e92fac1d6efa725c319248d279200',
        '0xf591e42dffe8e62c2085ccaadfe05f84d89d0c6',
        '0x9331fde4db7b9d9d1498c09d30149929f24cf9d5',
        '0x30c381033aa2830ceb0aa372c2e4d28f004b3db9',
        '0x69e423181f1d3e6bebf8ab88030c36da73785f26'
        )
    ),
    eth_gtc as (
    SELECT
        tx_hash,
        block_timestamp,
        voter,
        'ETH' as token,
        amount_usd
    FROM
        eth_time
    WHERE
        origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
        and eth_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
    ),

    all_votes as (
    SELECT * from eth_gtc
    union all 
    select * FROM dai
    union all
    select * from eth_direct
    ),

    unique_voter as (SELECT DISTINCT voter from all_votes)
    ,  
    transactions as (
    SELECT
          BLOCK_TIMESTAMP,
          TX_FEE,
          FROM_ADDRESS AS EOA,
          TO_ADDRESS AS COUNTERPARTY,
          ETH_VALUE,
          1 as BOOLEAN_OUT
        FROM
          optimism.core.fact_transactions tx
        INNER JOIN 
          unique_voter v 
            ON tx.FROM_ADDRESS = v.voter
        UNION ALL
        SELECT
          BLOCK_TIMESTAMP,
          TX_FEE,
          TO_ADDRESS AS EOA,
          FROM_ADDRESS AS COUNTERPARTY,
          ETH_VALUE,
          0 as BOOLEAN_OUT
        FROM
          optimism.core.fact_transactions tx
       INNER JOIN 
          unique_voter v 
            ON tx.TO_ADDRESS = v.voter)
SELECT
  EOA,
  COUNT(*) as n_tx,
  COUNT(DISTINCT(COUNTERPARTY)) as n_counterparty,
  SUM(ETH_VALUE) as eth_volume,
  SUM(BOOLEAN_OUT) as n_tx_out,
  n_tx - n_tx_out as n_tx_in,
  n_tx_out - n_tx_in as n_tx_diff_out_in,
  TIMESTAMPDIFF(MINUTE, MIN(BLOCK_TIMESTAMP), CURRENT_TIMESTAMP()) as age,
  TIMESTAMPDIFF(MINUTE, MIN(BLOCK_TIMESTAMP), MAX(BLOCK_TIMESTAMP)) as time_alive,
  CASE WHEN age = 0 THEN 0 ELSE n_tx / age END as tx_min,
  CASE WHEN time_alive = 0 THEN 0 ELSE n_tx / time_alive END as tx_min_alive,
  AVG(TX_FEE) as avg_tx_fee,
  STDDEV(TX_FEE) as std_tx_fee
FROM
  transactions
GROUP BY
  EOA
HAVING n_tx < 100
  ;
"""

In [40]:
url_tag_api = 'http://localhost:8000/inferFullSql'
headers = {'Content-type': 'application/json'}
response = requests.post(url_tag_api, headers=headers, json={"network": 'optimism', "sql": sql_template})
print(response.status_code)

200


In [41]:
response.content


b'{"result":"{\\"columns\\":[\\"eoa\\",\\"prediction\\"],\\"index\\":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,

In [42]:
df_res = pd.read_json(response.json()['result'], orient='split')
df_res.head(2)

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_8016\886112233.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_res = pd.read_json(response.json()['result'], orient='split')


,eoa,prediction
0,0x58fca18553046dcf2e11568ae58536ac97b516f4,0
1,0x31587b38778660a21cfa6631449073e3072a9a37,0


In [43]:
df_res.prediction.value_counts()

prediction
0    26612
1      753
Name: count, dtype: int64

In [44]:
df_res.rename(columns={'eoa': 'voter', 'prediction': 'bot'}, inplace=True)

In [ ]:
df_

In [47]:
df_res[df_res['bot']==1].to_csv(os.path.join(ODC_DATA_DIR, 'bot_gr18.csv'), index=False)

In [48]:
df_res[df_res['bot']==1].head()

,voter,bot
300,0x9b7c51e1091fd3672b1e4767e1dfab7d52d6486a,1
384,0xd5947af8d7ea77f50612acf513e901ce7c7f10f3,1
401,0x83319bd1ee98874c62cde0038c7af271c01498f3,1
436,0x43b75c45d61098857232f91d420e3dc399041760,1
447,0x0b3fb7ba63a9cb9ab9a5f9b45f9d9a5e3bfe8e1a,1
